# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2531046999713,
 'week52high': 186.53,
 'week52low': 135.23,
 'week52highSplitAdjustOnly': 184.4,
 'week52lowSplitAdjustOnly': 129.9,
 'week52change': 0.0387148939412753,
 'sharesOutstanding': 16154996126,
 'float': 0,
 'avg10Volume': 107781900,
 'avg30Volume': 84095472,
 'day200MovingAvg': 157.52,
 'day50MovingAvg': 169.21,
 'employees': 147128,
 'ttmEPS': 6.15,
 'ttmDividendRate': 0.9083410436887737,
 'dividendYield': 0.006214239067410944,
 'nextDividendDate': '',
 'exDividendDate': '2022-07-27',
 'nextEarningsDate': '2022-10-13',
 'peRatio': 25.30022003494773,
 'beta': 1.2536763833454485,
 'maxChangePercent': 58.41818107849055,
 'year5ChangePercent': 2.991346102651929,
 'year2ChangePercent': 0.3734895577152215,
 'year1ChangePercent': 0.019144742129054898,
 'ytdChangePercent': -0.14913333671953535,
 'month6ChangePercent': -0.05349022409868771,
 'month3ChangePercent': 0.1678149826306828,
 'month1ChangePercent': -0.1310705160814145,
 'day30Chan

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.019144742129054898

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [ ]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [9]:
from sqlalchemy import true


final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'                   
                ],
                index = my_columns
            ),
            ignore_index = True
        )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,133.36,-0.249144,N/A
1,AAL,14.03,-0.309973,N/A
2,AAP,172.60,-0.178947,N/A
3,AAPL,156.70,0.018616,N/A
4,ABBV,147.71,0.400480,N/A
...,...,...,...,...
95,CINF,101.54,-0.157330,N/A
96,CL,76.64,0.013301,N/A
97,CLX,147.09,-0.124081,N/A
98,CMA,79.15,0.102664,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [10]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CF,100.70,1.069434,N/A
1,APA,40.62,1.023758,N/A
2,CI,304.32,0.449734,N/A
3,ABBV,147.71,0.400480,N/A
4,ADM,88.58,0.383183,N/A
5,AZO,2189.60,0.356785,N/A
6,CAH,68.47,0.322213,N/A
7,ATO,114.80,0.299679,N/A
8,AIV,9.06,0.296392,N/A
9,ANTM,487.40,0.286760,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [13]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

10000000


In [14]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/Users/aidan.w/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CF,100.70,1.069434,1947
1,APA,40.62,1.023758,4827
2,CI,304.32,0.449734,644
3,ABBV,147.71,0.400480,1327
4,ADM,88.58,0.383183,2213
5,AZO,2189.60,0.356785,89
6,CAH,68.47,0.322213,2863
7,ATO,114.80,0.299679,1708
8,AIV,9.06,0.296392,21642
9,ANTM,487.40,0.286760,402


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [73]:
#take a basket of stocks with different time range
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    #print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'DISCA' or symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW':
            continue
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,134.17,N/A,-0.249064,N/A,-0.035723,N/A,0.135937,N/A,-0.021788,N/A,N/A
1,AAL,13.76,N/A,-0.308945,N/A,-0.172194,N/A,0.132679,N/A,-0.115518,N/A,N/A
2,AAP,169.50,N/A,-0.17485,N/A,-0.178627,N/A,0.017437,N/A,-0.201724,N/A,N/A
3,AAPL,152.10,N/A,0.018818,N/A,-0.054183,N/A,0.166471,N/A,-0.134338,N/A,N/A
4,ABBV,149.07,N/A,0.404259,N/A,-0.062888,N/A,0.046247,N/A,0.010983,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,115.92,N/A,-0.078842,N/A,-0.022875,N/A,0.067122,N/A,-0.023127,N/A,N/A
497,ZBH,116.66,N/A,-0.225123,N/A,-0.075348,N/A,0.094921,N/A,-0.030476,N/A,N/A
498,ZBRA,301.90,N/A,-0.516037,N/A,-0.28434,N/A,-0.012692,N/A,-0.169159,N/A,N/A
499,ZION,57.80,N/A,0.025421,N/A,-0.153877,N/A,0.129973,N/A,-0.015514,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [74]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        for i in range(len(hqm_dataframe[change_col])):
            if hqm_dataframe[change_col][i] is None:
                hqm_dataframe[change_col][i] = 0.00

        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.239521
1      0.153693
2      0.377246
3      0.694611
4      0.942116
         ...   
496    0.526946
497    0.289421
498     0.02994
499    0.702595
500    0.283433
Name: One-Year Return Percentile, Length: 501, dtype: object
0      0.634731
1      0.307385
2      0.287425
3      0.592814
4      0.580838
         ...   
496    0.654691
497    0.552894
498    0.093812
499    0.371257
500    0.279441
Name: Six-Month Return Percentile, Length: 501, dtype: object
0      0.758483
1      0.750499
2      0.333333
3      0.844311
4       0.44511
         ...   
496    0.518962
497    0.650699
498    0.213573
499    0.740519
500    0.279441
Name: Three-Month Return Percentile, Length: 501, dtype: object
0      0.848303
1      0.333333
2        0.0499
3      0.239521
4       0.94012
         ...   
496    0.842315
497    0.818363
498    0.101796
499    0.862275
500     0.50499
Name: One-Month Return Percentile, Length: 501, dtype: object


/var/folders/qt/h3qgx4hn4j9cdtgy975g7g2r0000gn/T/ipykernel_30052/2194248688.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe[change_col][i] = 0.00


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,134.17,N/A,-0.249064,0.239521,-0.035723,0.634731,0.135937,0.758483,-0.021788,0.848303,N/A
1,AAL,13.76,N/A,-0.308945,0.153693,-0.172194,0.307385,0.132679,0.750499,-0.115518,0.333333,N/A
2,AAP,169.50,N/A,-0.17485,0.377246,-0.178627,0.287425,0.017437,0.333333,-0.201724,0.0499,N/A
3,AAPL,152.10,N/A,0.018818,0.694611,-0.054183,0.592814,0.166471,0.844311,-0.134338,0.239521,N/A
4,ABBV,149.07,N/A,0.404259,0.942116,-0.062888,0.580838,0.046247,0.44511,0.010983,0.94012,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,115.92,N/A,-0.078842,0.526946,-0.022875,0.654691,0.067122,0.518962,-0.023127,0.842315,N/A
497,ZBH,116.66,N/A,-0.225123,0.289421,-0.075348,0.552894,0.094921,0.650699,-0.030476,0.818363,N/A
498,ZBRA,301.90,N/A,-0.516037,0.02994,-0.28434,0.093812,-0.012692,0.213573,-0.169159,0.101796,N/A
499,ZION,57.80,N/A,0.025421,0.702595,-0.153877,0.371257,0.129973,0.740519,-0.015514,0.862275,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [75]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,134.17,N/A,-0.249064,0.239521,-0.035723,0.634731,0.135937,0.758483,-0.021788,0.848303,0.620259
1,AAL,13.76,N/A,-0.308945,0.153693,-0.172194,0.307385,0.132679,0.750499,-0.115518,0.333333,0.386228
2,AAP,169.50,N/A,-0.17485,0.377246,-0.178627,0.287425,0.017437,0.333333,-0.201724,0.0499,0.261976
3,AAPL,152.10,N/A,0.018818,0.694611,-0.054183,0.592814,0.166471,0.844311,-0.134338,0.239521,0.592814
4,ABBV,149.07,N/A,0.404259,0.942116,-0.062888,0.580838,0.046247,0.44511,0.010983,0.94012,0.727046
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,115.92,N/A,-0.078842,0.526946,-0.022875,0.654691,0.067122,0.518962,-0.023127,0.842315,0.635729
497,ZBH,116.66,N/A,-0.225123,0.289421,-0.075348,0.552894,0.094921,0.650699,-0.030476,0.818363,0.577844
498,ZBRA,301.90,N/A,-0.516037,0.02994,-0.28434,0.093812,-0.012692,0.213573,-0.169159,0.101796,0.10978
499,ZION,57.80,N/A,0.025421,0.702595,-0.153877,0.371257,0.129973,0.740519,-0.015514,0.862275,0.669162


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [76]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe.reset_index(inplace = True, drop = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LB,80.28,N/A,2.321423,1.0,0.847842,1.0,0.221847,0.928144,0.080341,0.982036,0.977545
1,UNM,41.29,N/A,0.641736,0.964072,0.403836,0.992016,0.250205,0.948104,0.017251,0.948104,0.963074
2,COG,22.68,N/A,0.356622,0.924152,0.184237,0.954092,0.266591,0.952096,0.250445,1.0,0.957585
3,ALB,287.45,N/A,0.25898,0.882236,0.532902,0.996008,0.423146,0.994012,0.025354,0.956088,0.957086
4,FTI,9.25,N/A,0.269983,0.888224,0.294,0.982036,0.31446,0.97006,0.073748,0.976048,0.954092
5,NLSN,28.72,N/A,0.405289,0.944112,0.203849,0.96008,0.292919,0.966068,0.011573,0.942116,0.953094
6,IRM,57.57,N/A,0.296921,0.906188,0.156402,0.936128,0.220469,0.926148,0.034096,0.966068,0.933633
7,HRB,46.44,N/A,0.878123,0.986028,0.76558,0.998004,0.366541,0.984032,-0.049394,0.730539,0.924651
8,OXY,67.09,N/A,1.425873,0.998004,0.225462,0.97006,0.140043,0.780439,0.016721,0.946108,0.923653
9,HES,126.61,N/A,0.710096,0.978044,0.300862,0.984032,0.124006,0.726547,0.102964,0.988024,0.919162


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [77]:
portfolio_input()

In [78]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

/Users/aidan.w/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LB,80.28,2442,2.321423,1.0,0.847842,1.0,0.221847,0.928144,0.080341,0.982036,0.977545
1,UNM,41.29,4748,0.641736,0.964072,0.403836,0.992016,0.250205,0.948104,0.017251,0.948104,0.963074
2,COG,22.68,8645,0.356622,0.924152,0.184237,0.954092,0.266591,0.952096,0.250445,1.0,0.957585
3,ALB,287.45,682,0.25898,0.882236,0.532902,0.996008,0.423146,0.994012,0.025354,0.956088,0.957086
4,FTI,9.25,21197,0.269983,0.888224,0.294,0.982036,0.31446,0.97006,0.073748,0.976048,0.954092
5,NLSN,28.72,6827,0.405289,0.944112,0.203849,0.96008,0.292919,0.966068,0.011573,0.942116,0.953094
6,IRM,57.57,3405,0.296921,0.906188,0.156402,0.936128,0.220469,0.926148,0.034096,0.966068,0.933633
7,HRB,46.44,4222,0.878123,0.986028,0.76558,0.998004,0.366541,0.984032,-0.049394,0.730539,0.924651
8,OXY,67.09,2922,1.425873,0.998004,0.225462,0.97006,0.140043,0.780439,0.016721,0.946108,0.923653
9,HES,126.61,1548,0.710096,0.978044,0.300862,0.984032,0.124006,0.726547,0.102964,0.988024,0.919162


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [86]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [87]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [88]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', percent_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [89]:
writer.save()